In [1]:
import os
import csv

In [2]:
input_directory = '../data/raw/articles/'
output_directory = '../data/processed/fasta_sequences/'

In [3]:
def process_csv_to_fasta(csv_path, fasta_path):
    with open(csv_path, 'r') as csv_file, open(fasta_path, 'w') as fasta_file:
        reader = csv.reader(csv_file)
        next(reader)
        for row in reader:
            header = row[2].strip()
            sequence = row[3].strip()
            fasta_file.write(f">{header}\n{sequence}\n")

def process_directory(input_directory):
    for filename in os.listdir(input_directory):
        if filename.endswith('.csv'):
            csv_path = os.path.join(input_directory, filename)
            fasta_filename = os.path.splitext(filename)[0] + '.fasta'
            fasta_path = os.path.join(output_directory, fasta_filename)
            process_csv_to_fasta(csv_path, fasta_path)
            print(f"Processed {filename} -> {fasta_filename}")

In [4]:
process_directory(input_directory)

Processed AAA_ATPase_15-21_family.csv -> AAA_ATPase_15-21_family.fasta
Processed ABC_ATPase_PARIS.csv -> ABC_ATPase_PARIS.fasta
